# Step Three (B): Find Similar Entities via Adapted Wikipedia2vec most_similar()

We have now returned all of the entities we'll get from direct querying of the package. We must now use alternate measures to identify candidate entities and select from that pool.

# Note: This is the colab version used for trying to improve speed and use higher-dimensional vectors.

#### Import Packages

In [1]:
import os
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Progress bar
from tqdm import tqdm

In [2]:
!pip install wikipedia2vec

     |████████████████████████████████| 1.2MB 15.1MB/s 
     |████████████████████████████████| 276kB 49.6MB/s 
     |████████████████████████████████| 143kB 49.8MB/s 
  Created wheel for wikipedia2vec: filename=wikipedia2vec-1.0.4-cp36-cp36m-linux_x86_64.whl size=4581870 sha256=833e7e4c42b0bf6a5348c98bf32fdc95b15e2daf10bbd8c29ad005defc613f3a
  Stored in directory: /root/.cache/pip/wheels/16/e7/02/852c8ce366cc10adcf5d43c6471bbf926dd15c277578c13184
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=861510 sha256=394c5f5f7b78ea09a105b3bc325da10c3f1628f02bdbe170a1ddb77f515257f2
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
  Created wheel for mwparserfromhell: filename=mwparserfromhell-0.5.4-cp36-cp36m-linux_x86_64.whl size=183772 sha256=d64bd6e7b0e368abd1ec7389017bc345d0d91b3a9f8cfea532075a5e5524fccb
  Stored in directory: /root/.cache/pip/wheels/2a/76/d5/7088b941df3b362c45dd7912dd05314

In [3]:
# Package
from wikipedia2vec import Wikipedia2Vec

# Class to compare type
from wikipedia2vec.dictionary import Entity

In [4]:
# Download dimensional file from Wikipedia2vec website
!curl -O http://wikipedia2vec.s3.amazonaws.com/models/en/2018-04-20/enwiki_20180420_300d.pkl.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9921M  100 9921M    0     0  19.5M      0  0:08:26  0:08:26 --:--:-- 19.6M


In [5]:
!bzip2 -d enwiki_20180420_300d.pkl.bz2

In [6]:
!ls

enwiki_20180420_300d.pkl  sample_data


In [7]:
%%time
# Load unzipped pkl file with word embeddings
w2v = Wikipedia2Vec.load("enwiki_20180420_300d.pkl")

CPU times: user 48.8 ms, sys: 159 ms, total: 208 ms
Wall time: 2.39 s


## Load ACY Input Data

In [8]:
from google.colab import files
uploaded = files.upload()

Saving Aida-Conll-Yago-Input.csv to Aida-Conll-Yago-Input.csv


In [9]:
!ls

Aida-Conll-Yago-Input.csv  enwiki_20180420_300d.pkl  sample_data


In [10]:
# Load data
acy_input = pd.read_csv("Aida-Conll-Yago-Input.csv", delimiter=",")
acy_input.head(3)

,mention,full_mention,wikipedia_URL,wikipedia_page_ID,wikipedia_title,sentence_id,doc_id,congruent_mentions
0,B,EU,NaN,NaN,NaN,0,0,"['EU', 'German', 'British']"
1,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,0,0,"['EU', 'German', 'British']"
2,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,0,0,"['EU', 'German', 'British']"


In [11]:
# Re-name 
candidate_pools = acy_input.copy()

## Find most similar entity using Wikipedia2vec

We now turn to using a variation on Wikipedia2vec's `most_similar()` function to find, for entered words, the most similar entity. We do this as an added-layer, meaning only for those without an estimate, and for all full mentions, to compare performance.

In [12]:
# Normalize full_mentions to lower case for entry into most_similar() function
full_mention_norm = np.array([x.lower() for x in candidate_pools['full_mention']])
candidate_pools['full_mention_norm'] = full_mention_norm
candidate_pools.head(3)

,mention,full_mention,wikipedia_URL,wikipedia_page_ID,wikipedia_title,sentence_id,doc_id,congruent_mentions,full_mention_norm
0,B,EU,NaN,NaN,NaN,0,0,"['EU', 'German', 'British']",eu
1,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,0,0,"['EU', 'German', 'British']",german
2,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,0,0,"['EU', 'German', 'British']",british


In [13]:
### Test single full mention query time
start_time = time.time()

# Print word
search_word = candidate_pools['full_mention_norm'][2]
print("Search Word: ", search_word)

# Translate word into vector
# Handles multi-word mentions
search_word_list = search_word.split(" ")
search_word_vector = None
for word in search_word_list:
    try:
        vector = w2v.get_word_vector(str(word))
    except KeyError:
        print("This word is Out of Vocabulary (OOV) for Wikipedia2vec.")
        vector = None
        
    if search_word_vector is None:
        search_word_vector = vector
    else:
        search_word_vector += vector

if search_word_vector is not None:
    # Get most similar word
    count_similar = 500
    similar = w2v.most_similar_by_vector(search_word_vector, count_similar)

    # Retrieve only entities from word
    entities = []
    return_similar = 10
    for i in similar:
    #     print(type(i[0]))
        if isinstance(i[0], Entity):
            entities.append(i)
    #     if len(entities) == return_similar:
    #         break
    display(entities)
end_time = time.time()
print(f"Single Word Query Time: {round(end_time - start_time, 2)}s")

Search Word:  british


[(<Entity Derek George Cudmore>, 0.49873415),
 (<Entity HMS Birkenhead>, 0.48730087),
 (<Entity File:Turner, The Battle of Trafalgar (1822).jpg>, 0.48630852),
 (<Entity John Osbaldiston Field>, 0.48565567),
 (<Entity Henry Maynard Ball>, 0.48533762),
 (<Entity Edward John Cameron>, 0.48491052),
 (<Entity David Robert Barwick>, 0.48152602),
 (<Entity List of Commissioners of the Turks and Caicos Islands>, 0.47786582),
 (<Entity Starlight (TV series)>, 0.47318417),
 (<Entity List of colonial governors of the British Virgin Islands>,
  0.47302568),
 (<Entity Gordon Wesley Jewkes>, 0.4724612),
 (<Entity Rollo Pain>, 0.4719429),
 (<Entity Percivale Liesching>, 0.47097537),
 (<Entity Walter Wilkinson Wallace>, 0.46967942),
 (<Entity Millions (1937 film)>, 0.4696355),
 (<Entity United Kingdom>, 0.469226),
 (<Entity File:El Alamein 1942 - British infantry.jpg>, 0.469008),
 (<Entity Ernest Gordon Lewis>, 0.46749967),
 (<Entity The Windmill (1937 film)>, 0.4670006),
 (<Entity Paul Adam (English 

Single Word Query Time: 304.09s


In [ ]:
### Test single full mention query time on a mention with multiple words
start_time = time.time()

# Print word
search_word = candidate_pools['full_mention_norm'][51]
print("Search Word: ", search_word)

# Translate word into vector
# Handles multi-word mentions
search_word_list = search_word.split(" ")
search_word_vector = None
for word in search_word_list:
    try:
        vector = w2v.get_word_vector(str(word))
        
        if search_word_vector is None:
            search_word_vector = vector
        else:
            search_word_vector += vector

    except KeyError:
        print(f"\"{word}\" is Out of Vocabulary (OOV) for Wikipedia2vec.")

if search_word_vector is not None:
    # Get most similar word
    count_similar = 500
    similar = w2v.most_similar_by_vector(search_word_vector, count_similar)

    # Retrieve only entities from word
    entities = []
    return_similar = 10
    for i in similar:
    #     print(type(i[0]))
        if isinstance(i[0], Entity):
            entities.append(i)
    #     if len(entities) == return_similar:
    #         break
    display(entities)
end_time = time.time()
print(f"Single Word Query Time: {round(end_time - start_time, 2)}s")

Search Word:  welsh national farmers ' union
"'" is Out of Vocabulary (OOV) for Wikipedia2vec.


[(<Entity :Category:Wikipedians interested in the European Union>, 0.74048346),
 (<Entity Whiteheads RFC>, 0.7279615),
 (<Entity Ian Mackay (rugby league)>, 0.7100251),
 (<Entity E. Gwyndaf Evans>, 0.70472246),
 (<Entity Ystrad Rhondda RFC>, 0.7012322),
 (<Entity Abercynon RFC>, 0.70090365),
 (<Entity Caernarfon RFC>, 0.69320804),
 (<Entity Tredegar Ironsides RFC>, 0.69225544),
 (<Entity Fleur De Lys RFC>, 0.6915729),
 (<Entity Cwmgwrach RFC>, 0.6905903),
 (<Entity File:Harden NSW.PNG>, 0.6901128),
 (<Entity Betws RFC>, 0.6887361),
 (<Entity Dai Francis (trade union leader)>, 0.6886294),
 (<Entity Llandybie RFC>, 0.6878249),
 (<Entity Berwyn Rangers F.C.>, 0.6861579),
 (<Entity :Category:All Blacks>, 0.68473554),
 (<Entity :Eastern Conference (NHL)>, 0.6845522),
 (<Entity Markham RFC>, 0.6823692),
 (<Entity Bethesda RFC>, 0.6821802),
 (<Entity Fall Bay RFC>, 0.6820504),
 (<Entity Cwmtwrch RFC>, 0.6806142),
 (<Entity Neil Lashkari>, 0.6795225),
 (<Entity Joost Adriaan van Hamel>, 0.6789

Single Word Query Time: 1.9s


### Run Over Large Subset of Data

In [15]:
# Prepare output array
most_similar_entities = []
most_similar_scores = []
get_similar_candidate_pool = []
get_similar_candidate_scores = []

# Track metrics
success_word_query = 0
oov_errors = 0
start_time = time.time()

# Provide filter ability
size = 500

for mention in tqdm(candidate_pools['full_mention_norm'][:size]):
    
    # Translate word into vector
    # Handles multi-word mentions
    search_word_list = mention.split(" ")
    search_word_vector = None
    for word in search_word_list:
        try:
            vector = w2v.get_word_vector(str(word))
            
            if search_word_vector is None:
                search_word_vector = vector
            else:
                search_word_vector += vector
                
        except KeyError:
            oov_errors += 1

    # Save candidate pool
    candidate_pool = []
    candidate_scores = []
    
    if search_word_vector is not None:
        success_word_query += 1
        
        # Search most similar words/entities from found word
        # Retrieve 500 most similar to test large coverage
        similars = w2v.most_similar_by_vector(search_word_vector, 500)

        # Retrieve most similar entity
        most_similar = None
        for s in similars:
            if isinstance(s[0], Entity):
                candidate_pool.append(s[0].title)
                candidate_scores.append(s[1])
                if most_similar is None:
                    most_similar = s
                
    # Save lists
    get_similar_candidate_pool.append(candidate_pool)
    get_similar_candidate_scores.append(candidate_scores)
    
    if most_similar is not None:
        most_similar_entities.append(most_similar[0].title)
        most_similar_scores.append(most_similar[1])
    else:
        most_similar_entities.append(None)
        most_similar_scores.append(None)

    
print("Successfully Found Words: ", round(success_word_query/size*100,3),"%")
print("Out-of-Vocabulary Issues: ", round(oov_errors/size*100,3),"%")
execution_time = time.time() - start_time
print("Execution time: ", round(execution_time, 3),"s")

100%|██████████| 500/500 [41:04<00:00,  4.93s/it]

Successfully Found Words:  96.8 %
Out-of-Vocabulary Issues:  7.4 %
Execution time:  2464.37 s


In [16]:
# Append to dataframe
mini_df = candidate_pools[:size].copy()
mini_df['preds_w2v_mostsimilar'] = most_similar_entities
mini_df['score_w2v_mostsimilar'] = most_similar_scores
mini_df['candidate_pool_mostsimilar'] = get_similar_candidate_pool
mini_df['candidate_scores_mostsimilar'] = get_similar_candidate_scores
mini_df.head(3)

,mention,full_mention,wikipedia_URL,wikipedia_page_ID,wikipedia_title,sentence_id,doc_id,congruent_mentions,full_mention_norm,preds_w2v_mostsimilar,score_w2v_mostsimilar,candidate_pool_mostsimilar,candidate_scores_mostsimilar
0,B,EU,NaN,NaN,NaN,0,0,"['EU', 'German', 'British']",eu,European Union,0.668622,"[European Union, Directorate-General for Trade...","[0.668622, 0.61401534, 0.6123832, 0.60006154, ..."
1,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,0,0,"['EU', 'German', 'British']",german,1856 in Germany,0.597120,"[1856 in Germany, 1860 in Germany, 1862 in Ger...","[0.5971198, 0.58756816, 0.57571477, 0.5744798,..."
2,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,0,0,"['EU', 'German', 'British']",british,Derek George Cudmore,0.498734,"[Derek George Cudmore, HMS Birkenhead, File:Tu...","[0.49873415, 0.48730087, 0.48630852, 0.4856556..."


In [17]:
# Estimate length of time to run over full dataset
print("Estimated Duration for Full Dataset: ",\
     round((len(candidate_pools)/size)*execution_time/60/60,2), " hours")

Estimated Duration for Full Dataset:  40.13  hours


## Calculate Accuracy of Most Similar Entity Predictions

In [18]:
# Calculate accuracy
accurate_predictions = (mini_df['preds_w2v_mostsimilar'] == mini_df['wikipedia_title']).sum()
print("****************************")
print(f"Predictive Accuracy: {round(accurate_predictions / len(mini_df) * 100, 3)}%")
print("****************************")

****************************
Predictive Accuracy: 31.6%
****************************


In [19]:
# Calculate percentage of candidate pools with the correct answer present
# Use Wikipedia Title
# Necessary to determine if shuffling pool could even get the right answer
response_present = [mini_df['wikipedia_title'][i] in mini_df['candidate_pool_mostsimilar'][i] for i in range(len(mini_df))]
print(f"Correct answer is present in {round(sum(response_present) / len(mini_df) * 100, 3)}% of generated candidate pools via adapted Wikipedia2vec's most_similar() method.")

Correct answer is present in 43.2% of generated candidate pools via adapted Wikipedia2vec's most_similar() method.



## Save predictive dataframe for input to next step

In [20]:
#Final DF
mini_df.head(10)

,mention,full_mention,wikipedia_URL,wikipedia_page_ID,wikipedia_title,sentence_id,doc_id,congruent_mentions,full_mention_norm,preds_w2v_mostsimilar,score_w2v_mostsimilar,candidate_pool_mostsimilar,candidate_scores_mostsimilar
0,B,EU,NaN,NaN,NaN,0,0,"['EU', 'German', 'British']",eu,European Union,0.668622,"[European Union, Directorate-General for Trade...","[0.668622, 0.61401534, 0.6123832, 0.60006154, ..."
1,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,0,0,"['EU', 'German', 'British']",german,1856 in Germany,0.597120,"[1856 in Germany, 1860 in Germany, 1862 in Ger...","[0.5971198, 0.58756816, 0.57571477, 0.5744798,..."
2,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,0,0,"['EU', 'German', 'British']",british,Derek George Cudmore,0.498734,"[Derek George Cudmore, HMS Birkenhead, File:Tu...","[0.49873415, 0.48730087, 0.48630852, 0.4856556..."
3,B,Peter Blackburn,NaN,NaN,NaN,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",peter blackburn,Heywood United F.C.,0.608454,"[Heywood United F.C., Adam Jackson, Blackburn ...","[0.60845387, 0.59419054, 0.5897064, 0.584375, ..."
4,I,Peter Blackburn,NaN,NaN,NaN,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",peter blackburn,Heywood United F.C.,0.609467,"[Heywood United F.C., Blackburn Rovers F.C., A...","[0.6094674, 0.6015017, 0.5913171, 0.58464885, ..."
5,B,BRUSSELS,http://en.wikipedia.org/wiki/Brussels,3708.0,Brussels,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",brussels,Brussels,0.869070,"[Brussels, Bordet railway station, Timeline of...","[0.8690704, 0.6816344, 0.67974824, 0.6619611, ..."
6,B,European Commission,http://en.wikipedia.org/wiki/European_Commission,9974.0,European Commission,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",european commission,:Category:Wikipedians interested in the Europe...,0.662552,[:Category:Wikipedians interested in the Europ...,"[0.6625522, 0.55637425, 0.5504225, 0.5464806, ..."
7,I,European Commission,http://en.wikipedia.org/wiki/European_Commission,9974.0,European Commission,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",european commission,:Category:Wikipedians interested in the Europe...,0.633569,[:Category:Wikipedians interested in the Europ...,"[0.6335695, 0.55570084, 0.53732044, 0.52874714..."
8,B,German,http://en.wikipedia.org/wiki/Germany,11867.0,Germany,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",german,1856 in Germany,0.597120,"[1856 in Germany, 1860 in Germany, 1862 in Ger...","[0.5971198, 0.58756816, 0.57571477, 0.5744798,..."
9,B,British,http://en.wikipedia.org/wiki/United_Kingdom,31717.0,United Kingdom,1,0,"['Peter Blackburn', 'BRUSSELS', 'European Comm...",british,Derek George Cudmore,0.498734,"[Derek George Cudmore, HMS Birkenhead, File:Tu...","[0.49873415, 0.48730087, 0.48630852, 0.4856556..."


In [22]:
# Save dataframe
mini_df.to_csv("wikipedia2vec_most_similar_500n_300d.csv", index=False)

In [23]:
# Download from Colab
files.download("wikipedia2vec_most_similar_500n_300d.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>